<a href="https://colab.research.google.com/github/Lufec/ML_Pima_Indians/blob/main/1_Bagging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pima Indians Diabetes - Bagging e Árvore de Decisão

In [ ]:
!pip install shap
!pip install lime
!pip install eli5

     |████████████████████████████████| 327kB 11.4MB/s 
  Created wheel for shap: filename=shap-0.37.0-cp36-cp36m-linux_x86_64.whl size=463904 sha256=f5abeef5119116195602ed4167a9589b51ceb448696452d0356cae13ca3cf130
  Stored in directory: /root/.cache/pip/wheels/df/ad/b0/aa7815ec68850d66551ef618095eccb962c8f6022f1d3dd989
Successfully built shap
     |████████████████████████████████| 276kB 12.2MB/s 
  Created wheel for lime: filename=lime-0.2.0.1-cp36-none-any.whl size=283845 sha256=ccd332a114f1f9d7febcfbf7208a8b679baafd75d3293be5dcc2a5071941570e
  Stored in directory: /root/.cache/pip/wheels/4c/4f/a5/0bc765457bd41378bf3ce8d17d7495369d6e7ca3b712c60c89
Successfully built lime
ERROR: Operation cancelled by user
     |████████████████████████████████| 112kB 13.0MB/s 


In [ ]:
from random import randint
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import kurtosis
from scipy.stats import skew
from scipy.stats import probplot
from scipy.stats import shapiro
from collections import Counter
from itertools import groupby
import pylab
import pandas as pd
from sklearn import preprocessing
from sklearn.impute import SimpleImputer


In [ ]:
np.set_printoptions(precision = 5,edgeitems = 11, linewidth = 200, suppress = True)

### Visualizando dados do dataset Pima

In [ ]:
df = pd.read_excel("/content/Diabetes_Original.xlsx", usecols = "A:I", na_filter = False)
df_copy = df
df.head(5)

## Analisando e balanceando o nosso target

In [ ]:
## 500 não diabéticos e 268 diabéticos
Counter(df['Diagnóstico'])

In [ ]:
n_diabeticos_bolean = df['Diagnóstico']==1
n_diabeticos = df[n_diabeticos_bolean]

In [ ]:
#indexs = np.random.choice(range(1,500),268)
#new_n_diabeticos = n_diabeticos.iloc[indexs]

In [ ]:
#df = pd.concat([new_n_diabeticos, df[df['Diagnóstico']==2]])

## Informações do dataset

In [ ]:
df.info()

## Busca por 0's no dataset

In [ ]:
(df == 0).astype(int).sum()

Variáveis em que valor 0 não apresentam sentido clínico: Glicose, Pressão, Pele, Insulina, Massa Corpórea.
Resolução: substituir os 0's pelas medianas de cada atributo

In [ ]:
imp_mean = SimpleImputer(missing_values=0, strategy='median')
features = ['Glicose','Pressão','Massa Corpórea','Insulina','Pele']   #Ainda não tô achando bom usar insulina  e pele aqui
imp_mean.fit(df[features]) 
df[features] = imp_mean.transform(df[features])
df

Para os casos de Insulina e pele....

In [ ]:
#'Pele',"Insulina"....
df_p0 = df[df["Pele"] == 0]
df_p0c1 = df_p0[df_p0["Diagnóstico"] == 1]
df_p0c2 = df_p0[df_p0["Diagnóstico"] == 2]
df_p1 = df[df["Pele"] != 0]
df_p1c1 = df_p1[df_p1["Diagnóstico"] == 1]
df_p1c2 = df_p1[df_p1["Diagnóstico"] == 2]
df_i0 = df[df["Insulina"] == 0]
df_i0c1 = df_i0[df_i0["Diagnóstico"] == 1]
df_i0c2 = df_i0[df_i0["Diagnóstico"] == 2]
df_i1 = df[df["Insulina"] != 0]
df_i1c1 = df_i1[df_i1["Diagnóstico"] == 1]
df_i1c2 = df_i1[df_i1["Diagnóstico"] == 2]


## Separar Dados em atributos e classe, separar as labels de ambos

In [ ]:
df_var = df.drop(columns = "Diagnóstico")
df_cls = df['Diagnóstico']
features = list(df_var)
classe = ['Diagnóstico']

In [ ]:
Counter(df_cls)

## Box-plot das variáveis

In [ ]:
df_var.boxplot()

## Ocorrência de outlies por variável

In [ ]:
for column in df:
  print(column)
  z = np.abs(stats.zscore(df[column]))
  print(np.where(z > 3))

## Normalizando as variáveis

In [ ]:
df_var_normz = (df_var-df_var.mean())/df_var.std()

In [ ]:
#z_scores = stats.zscore(df_var_normz)
#abs_z_scores = np.abs(z_scores)
#filtered_entries = (abs_z_scores < 3). all(axis=1)
#df_var_normz = df_var_normz[filtered_entries].reset_index(drop = True)
#df_cls = df_cls[filtered_entries].reset_index(drop = True)

## Box-plot das variáveis normalizadas

In [ ]:
df_var_normz.boxplot()

## Distribuição das variáveis normalizadas

In [ ]:
plt.figure(figsize=(15,15))
df_var_normz.hist(bins = 50)
plt.show

## Q-Q plot das variáveis normalizadas

In [ ]:
fig = plt.figure(figsize = (15,10))

ax1 = plt.subplot(241)
res = stats.probplot(df_var_normz["Gravidez"],plot = plt)
ax2 = plt.subplot(242)
res = stats.probplot(df_var_normz["Glicose"],plot = plt)
ax3 = plt.subplot(243)
res = stats.probplot(df_var_normz["Pressão"],plot = plt)
ax4 = plt.subplot(244)
res = stats.probplot(df_var_normz["Pele"],plot = plt)
ax5 = plt.subplot(245)
res = stats.probplot(df_var_normz["Insulina"],plot = plt)
ax6 = plt.subplot(246)
res = stats.probplot(df_var_normz["Massa Corpórea"],plot = plt)
ax7 = plt.subplot(247)
res = stats.probplot(df_var_normz["Genealogia"],plot = plt)
ax8 = plt.subplot(248)
res = stats.probplot(df_var_normz["Idade"],plot = plt)
							
plt.show()

## Teste de Normalidade

In [ ]:
# normality test
for column in df_var_normz:
  stat, p = shapiro(df_var_normz[column])
  print(column)
  print('Statistics=%.3f, p=%.3f' % (stat, p))
  # interpret
  alpha = 0.05
  if p > alpha:
	  print('Variavel aparenta ser Gaussiana (Nao rejeita H0)')
  else:
	  print('Variavel não aparenta ser Gaussiana (Rejeita H0)')

## Análise de Correlação

In [ ]:
corr=np.corrcoef(df_var_normz,rowvar = False)
corr

In [ ]:
import seaborn as sns
sns.heatmap(corr)

## Análise de Redução de dimensionalidade

In [ ]:
df_aval, df_avet = np.linalg.eig(corr)
print("Autovetores: \n",df_avet)
print("Autovalores: \n",df_aval)

In [ ]:
percent = df_aval/sum(df_aval) # d)
print(percent)

In [ ]:
#df_norm_decorr = np.dot(df_var_normz,df_avet).T

In [ ]:
#df_decorr_cls = pd.DataFrame(df_norm_decorr.T)
#df_cls = pd.DataFrame(df_cls)
#df_decorr_cls = pd.concat([df_decorr_cls,df_cls],axis = 1).reset_index(drop = True)

In [ ]:
#df_decorr_cls.shape

In [ ]:
#precisa 232 diabeticos
df_diab = df[df['Diagnóstico'] == 2].drop(columns = 'Diagnóstico')
df_dmax = df_diab.max()
df_dmin = df_diab.min()
df_dm = df_diab.mean()
df_dstd = df_diab.std()
cov = df_diab.cov()
cov

In [ ]:
#from scipy.stats import norm
#from scipy.linalg import cholesky
#num_samples = 232
#x = norm.rvs(size=(8, num_samples))

In [ ]:
#c = cholesky(cov, lower = True)
#evals, evecs = np.linalg.eig(cov)
 #Construct c, so c*c^T = r.
#c = np.dot(evecs, np.diag(np.sqrt(evals)))
#y = np.dot(c, x)
#np.set_printoptions(suppress=True)
#np.cov(y)

In [ ]:
import scipy.stats
def my_distribution(min_val, max_val, mean, std):
    scale = max_val - min_val
    location = min_val
    # Mean and standard deviation of the unscaled beta distribution
    unscaled_mean = (mean - min_val) / scale
    unscaled_var = (std / scale) ** 2
    # Computation of alpha and beta can be derived from mean and variance formulas
    t = unscaled_mean / (1 - unscaled_mean)
    beta = ((t / unscaled_var) - (t * t) - (2 * t) - 1) / ((t * t * t) + (3 * t * t) + (3 * t) + 1)
    alpha = beta * t
    # Not all parameters may produce a valid distribution
    if alpha <= 0 or beta <= 0:
        raise ValueError('Cannot create distribution for the given parameters.')
    # Make scaled beta distribution with computed parameters
    return scipy.stats.beta(alpha, beta, scale=scale, loc=location)

In [ ]:
my_dist = []

for i in range(8):
  
  my_dist.append(my_distribution(df_dmin[i], df_dmax[i], df_dm[i], df_dstd[i]))


In [ ]:
sample = []

for i in range(8):
  sample.append(my_dist[i].rvs(size=232).T)

new_data = pd.DataFrame(sample).transpose()
new_data = new_data.rename(columns = {0:"Gravidez", 1:"Glicose",2:"Pressão",
                                      3:"Pele",4:"Insulina",5:"Massa Corpórea",6:"Genealogia",7:"Idade"})
new_data.cov()

In [ ]:
new_data = (new_data-new_data.mean())/new_data.std()

In [ ]:
new_data.insert(8,column = "Diagnóstico", value = 2)

In [ ]:
df_cls = pd.DataFrame(df_cls)
df_normz = pd.concat([df_var_normz,df_cls],axis = 1).reset_index(drop = True)
df_normz = pd.concat([df_normz,new_data]).reset_index(drop = True)
df_normz

In [ ]:
df_normz["Diagnóstico"].value_counts()

In [ ]:
sns.heatmap(df_normz.drop(columns = "Diagnóstico").corr())

In [ ]:
df.corr()

In [ ]:
df_normz.corr()

## Divisão dos dados de treino e teste (80% e 20%)

In [ ]:
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_normz.drop(columns = "Diagnóstico"), df_normz["Diagnóstico"], test_size=0.2)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


## Exportando dados para futuras análises

In [ ]:
df_treino = X_train.copy()
df_treino.insert(8,column = "Diagnóstico",value = y_train)
df_treino = df_treino.reset_index(drop = True)
df_treino.to_csv(r'dfTreino20.csv', index = False, header=True)

In [ ]:
df_teste = X_test.copy()
df_teste.insert(8,column = "Diagnóstico",value = y_test)
df_teste=df_teste.reset_index(drop = True)
df_teste.to_csv(r'dfTeste20.csv', index = False, header=True)

In [ ]:
X_train

In [ ]:
dfTreino = pd.read_csv("/content/dfTreino20.csv")
dfTeste = pd.read_csv("/content/dfTeste20.csv")

In [ ]:
dfTreino["Diagnóstico"].value_counts()

2    400
1    400
Name: Diagnóstico, dtype: int64

In [ ]:
x_train = dfTreino.drop(columns = "Diagnóstico")
y_train = dfTreino['Diagnóstico']
x_test =  dfTeste.drop(columns = "Diagnóstico")
y_test =  dfTeste['Diagnóstico']

## Utilizando o algoritmo de árvore de decisão do scikit-learn

In [ ]:
from sklearn import tree
from sklearn.metrics import confusion_matrix,accuracy_score,average_precision_score
classf = tree.DecisionTreeClassifier(criterion = 'entropy', max_depth = 9)
classf = classf.fit(x_train, y_train)
classf

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=9, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

### Matriz de confusão do modelo nos dados de teste

In [ ]:
result = classf.predict(x_test)
confusion_matrix(y_test, result)

array([[84, 16],
       [26, 74]])

### Acurácia para os dados de teste

In [ ]:
accuracy_score(y_test,result)

0.79

### Média de precisão para os dados de teste

In [ ]:
average_precision_score(y_test,result)

0.44641304347826083

## Gráfico de importância (SHAP) - Treino
 0   Gravidez        
 1   Glicose         
 2   Pressão          
 3   Pele            
 4   Insulina         
 5   Massa Corpórea  
 6   Genealogia      
 7   Idade           
 8   Diagnóstico

In [ ]:
import shap
shap.initjs()
shap_values = shap.TreeExplainer(classf).shap_values(x_train)
shap.summary_plot(shap_values, x_train)

## Gráfico de importância (SHAP) - Teste
 0   Gravidez        
 1   Glicose         
 2   Pressão          
 3   Pele            
 4   Insulina         
 5   Massa Corpórea  
 6   Genealogia      
 7   Idade           
 8   Diagnóstico

In [ ]:
shap.initjs()
shap_values = shap.TreeExplainer(classf).shap_values(x_test)
shap.summary_plot (shap_values, x_test)

## Gráfico de árvore do modelo

In [ ]:
import pydotplus
import matplotlib.pyplot as plt
import matplotlib.image as pltimg
import graphviz 

In [ ]:
features = ['Gravidez','Glicose','Pressão','Pele','Insulina','Massa Corpórea','Genealogia','Idade']
data = tree.export_graphviz(classf, out_file=None, feature_names=features, class_names=["1","2"])
graph = pydotplus.graph_from_dot_data(data)
graph.write_png('mydecisiontree.png')

fig, ax = plt.subplots(figsize=(15, 15))
img=pltimg.imread('mydecisiontree.png')
ax.imgplot = plt.imshow(img)
plt.show()

## Testando o modelo com bagging

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import BaggingClassifier

### Scores do modelo de treinamento com bagging

In [ ]:
dstree = tree.DecisionTreeClassifier(criterion = 'entropy', max_depth = 10)
model = BaggingClassifier(dstree, n_estimators=21)
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
n_scores = cross_val_score(model, x_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
print(n_scores)
print('Média e desvio padrão dos Scores de validação: %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)))

[0.925   0.9125  0.9     0.93125 0.90625 0.90625 0.8875  0.90625 0.86875 0.9625  0.89375 0.875   0.90625 0.9125  0.90625]
Média e desvio padrão dos Scores de validação: 0.907 (0.022)


In [ ]:
model = model.fit(x_train, y_train.values.ravel())

In [ ]:
resultBag = model.predict(x_test)

### Matriz de confusão do modelo nos dados de teste

In [ ]:
confusion_matrix(y_test, resultBag)

array([[85, 15],
       [14, 86]])

In [ ]:

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [ ]:
print(confusion_matrix(y_test,resultBag))
print(classification_report(y_test,resultBag))

[[85 15]
 [14 86]]
              precision    recall  f1-score   support

           1       0.86      0.85      0.85       100
           2       0.85      0.86      0.86       100

    accuracy                           0.85       200
   macro avg       0.86      0.85      0.85       200
weighted avg       0.86      0.85      0.85       200



### Acurácia para os dados de teste

In [ ]:
accuracy_score(y_test, resultBag)

0.855

### Média de precisão para os dados de teste

In [ ]:
average_precision_score(y_test, resultBag)

0.4496

# Conclusão

## Introdução
Neste notebook foi utilizado para classificação de pessoas com diabetes ou não o algoritmo de árvore de decisão. 

### Os dados
Os dados utilizados foram os do dataset pima que no decorrer das análises foi manipulado para uma redução de dimensionalidade tanto pelo descorrelacionamento quanto pelo balanceamento das variáveis de resposta, além disso os dados foram normalizados com finalidade de melhor desempenho do modelo.

### O modelo
Foi testado o algoritmo de árvore de decisão e também foi testada a possibilidade de bagging no modelo, em ambos os casos o modelo teve comportamento igual chegando a uma acurácia de 83%, tendo como as três variáveis mais importantes para predição: Gravidez, Idade e Glicose segundo o indice de SHAP.

Embora a acurácia seja boa, por haver baixo volume de dados o modelo não tem uma média de precisão muito grande, chegando a aproximadamente 45%.

